In [1]:
import pandas as pd

In [7]:
YearJoined = pd.read_csv('./data/year_joined.csv')
YearJoined[:3]

,user,userStats,yearJoined
0,0,silver,2014
1,1,silver,2015
2,2,silver,2016


In [9]:
YearJoined.groupby('user').count().groupby('userStats').count()

,yearJoined
userStats,
1,1000


In [80]:
emails = pd.read_csv('./data/emails.csv')
print(email.shape)
emails[:3]

(25488, 3)


,emailsOpened,user,week
0,3.0,1.0,2015-06-29 00:00:00
1,2.0,1.0,2015-07-13 00:00:00
2,2.0,1.0,2015-07-20 00:00:00


In [81]:
emails[emails.emailsOpened < 1]

,emailsOpened,user,week


In [82]:
emails[emails.user == 998][:10]

,emailsOpened,user,week
25464,1.0,998.0,2017-12-04 00:00:00
25465,3.0,998.0,2017-12-11 00:00:00
25466,3.0,998.0,2017-12-18 00:00:00
25467,3.0,998.0,2018-01-01 00:00:00
25468,3.0,998.0,2018-01-08 00:00:00
25469,2.0,998.0,2018-01-15 00:00:00
25470,3.0,998.0,2018-01-22 00:00:00
25471,2.0,998.0,2018-01-29 00:00:00
25472,3.0,998.0,2018-02-05 00:00:00
25473,3.0,998.0,2018-02-12 00:00:00


In [83]:
import datetime

emails.week = [datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S') for x in emails.week]

(max(emails[emails.user == 998].week) - min(emails[emails.user == 998].week)).days/7

25.0

In [84]:
emails[emails.user == 998].shape

(24, 3)

In [87]:
complete_idx = pd.MultiIndex.from_product((set(emails.week),set(emails.user)))

# index 설정하는거 익숙해지자
all_email = emails.set_index(['week', 'user']).reindex(complete_idx, fill_value=0).reset_index()
all_email.columns = ['week', 'user', 'EmailsOpened']

In [89]:
all_email[all_email.user == 998].sort_values('week')

,week,user,EmailsOpened
84083,2015-02-09,998.0,0.0
2155,2015-02-16,998.0,0.0
55516,2015-02-23,998.0,0.0
51743,2015-03-02,998.0,0.0
15091,2015-03-09,998.0,0.0
...,...,...,...
85700,2018-04-30,998.0,3.0
70069,2018-05-07,998.0,3.0
7545,2018-05-14,998.0,3.0
11318,2018-05-21,998.0,3.0


In [90]:
cutoff_dates = emails.groupby('user').week.agg(['min', 'max']).reset_index()

In [92]:
import warnings
warnings.filterwarnings('ignore')

for _, row in cutoff_dates.iterrows():
    user = row['user']
    start_date = row['min']
    end_date = row['max']

    all_email.drop(all_email[all_email.user == user][all_email.week < start_date].index, inplace=True)
    all_email.drop(all_email[all_email.user == user][all_email.week > end_date].index, inplace=True)